<a name="cell-solving"></a>

### 4. [Algorithmic Speed](#cell-sovling-Axb) / [Direct Methods](#cell-sovling-Axb) 
1. [Computational Speed](#cell-computational-speed)
    1. [Optimized Operations](#cell-operation-optimization)
    2. [Operation Counting](#cell-operation-counting)
        - [Sequential summation VS the `fan` method](#cell-operation-counting-fan)
    3. ["Big O" Algorithmic Complexity](#cell-sovling-bigO)
        - [$O(n) \text{ and } O(1)$](#cell-sovling-bigO-On-O1)
        - [$O(n^2) \text{ and } O(n^3)$](#cell-sovling-bigO-On2-On3)
            - [A Counting Example with Convolutions](#cell-FPN-counting)
        - [$O(n\log(n)) \text{ and } O(n^{\log_2 7})$](#cell-sovling-bigO-Onlogn) 
            - [Fast Fourier Transform Preview](#cell-sovling-bigO-Onlogn)

2. [Direct Methods finding](#cell-sovling-direct-methods) [$X\hat \beta \approx y \text{ by solving } \overset{A\, x}{X^TX \hat \beta} = \overset{b}{X^T y}$](#cell-sovling-direct-methods): (and see also `STA410_W24_Week5_Extra_MoreLeastSquares.ipynb`)
    1. ["Big O" Algorithmic Complexity](#cell-sovling-direct-methods-big0)


In [2]:
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib

<a name="cell-sovling-Axb"></a>

# 4 $Ax \approx b$ / Algorithmic Speed ([Return to TOC](#cell-solving))

---

There is a often tradeoff between numerical precision and computational efficiency.

> The `fahan` summation method and the recursive formula of the variance calculation from the `STA410_W24_Week4_Homework_AdditionVariance.ipynb` notebook were good examples of this.  They both performed extra numerical operations in an attempt to reduce ***roundoff error*** in the sequential operations.

Before looking more closely at the tradeoff between numerical precision and computational efficiency, let's first consider computational efficiency alone.

<a name="cell-computational-speed"></a>

## 4.0 Computational Speed ([Return to TOC](#cell-solving))

---

There are three factors driving the computational speed of any algorithm:

0. How much time does each actual operation in the algorithm take?
1. How many such operations are required to complete the algorithm for a given problem input size?
2. How do required operations scale as the problem size grows?

<a name="cell-operation-optimization"></a>

## 4.1.0 Optimized Operations ([Return to TOC](#cell-solving))

---


Following up on the Digital VS Analog Computing videos in the `STA410_W24_Week4_Extra_AnalogVsDigital_BitstringArithmatic_GracefulUnderflow.ipynb` notebook, it is exactly the case that computational hardware is built to perform specific operations and is then subsequently optimized for exactly that task. Subsequently, data types and operations on them which directly map onto optimized computational hardware processes can be extremely fast.

This is demonstrated below with the `np.array` data type which can process certain operations up to 100 times faster than a `list` data type.  

In [ ]:
# a "vector" -- mapping directly onto "vectorized" hardware computations
x_np_array = stats.norm().rvs(10000)
# a "linked list" -- which must be traversed sequentially as A->B->C->D->etc.
# and therefore does not simply map right onto a computational hardware operation
n, x_list = len(x_np_array), list(x_np_array)

In [ ]:
%timeit sum(x_list)

In [ ]:
# ultra optimized
%timeit x_np_array.sum()
# ...doesn't do "memory walk" traversing a "linked list" like A->B->C->D->etc.
# ...just gets mapped onto a computational hardware operation which then executes

In [ ]:
# and look how optimized the `x_np_array.sum()` operation is compared to the following
%timeit x_np_array.sum()/n
# summing 10000 is so much faster than dividing by n...

In [ ]:
# Interestingly, x_np_array.sum() is about twice as fast as x_np_array.mean()
%timeit x_np_array.mean()
# Presumably x_np_array.mean() is dividing every number by n before summing them
# which means it requires twice as many computations as x_np_array.sum()
# but this will minimize roundoff overall by accumulating values closer to 0

# This is another good example of the numerical accuracy / computational efficiency tradeoff

In [ ]:
# so instead of x_np_array.sum() the following might be more accuarte
%timeit x_np_array.mean()*n
# but for the example data used here x_np_array.mean()*n == x_np_array.sum()

10.7 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


<a name="cell-operation-counting"></a>

## 4.1.1 Operation Counting ([Return to TOC](#cell-solving))

---

Numeric operations are one basic kind of operations performed in an algorithm. Accessing and placing data in memory is another.

Recall the "two pass" and "single pass" sum of squares algorithms


$$\sum_{i=1}^n(x_i-\bar x)^2 = \underset{\text{first computes } \bar x}{\underbrace{\sum_{i=1}^n x_i^2 - n \bar x^2}_{\text{two pass algorithm}}} = \underset{\text{$\sum_{i=1}^{k-1 \rightarrow k} x_i^2$ and $\sum_{i=1}^{k-1\rightarrow k} x_i$ at step $k$}}{\underbrace{\sum_{i=1}^n x_i^2 - \frac{1}{n} \left(\sum_{i=1}^n x_i\right)^2}_{\text{single pass algorithm increments}}}$$

While the number of arithmetic operations required by each method is the same, the number of memory access operations required by each is different; namely, the two pass algorithm must access the data in memory *twice* as many times as the single pass algorithm. 

> The recursive formulation of the above calcutions does entail additional computations intended to minimize ***roundoff error***, which is more similar to the algorithm employed by `np.var()` function. So the `np.var()` function uses minimal memory access operations, but about twice as many numeric operations as the two "naive" algorithms above.

In [ ]:
def ss_two_pass(x,n):
  sumx2 = x[0]**2
  for i in range(1, n):
    sumx2 = sumx2 + x[i]**2
  sumx = x[0]
  for i in range(1, n):
    sumx = sumx + x[i]
  return sumx2-(sumx**2)/n

def ss_single_pass(x,n):
  sumx = x[0]
  sumx2 = sumx**2
  for i in range(1, n):
    xi = x[i]
    sumx2 = sumx2+xi**2
    sumx = sumx+xi
  return sumx2-(sumx**2)/n

# just numerical operations without accessing memory
def ss_numerical_operations(x,n):
  sumx = x[0]
  sumx2 = sumx**2
  for i in range(1, n):
    sumx2+sumx2**2
    sumx+sumx
  return sumx2-(sumx**2)/n

In [ ]:
%timeit ss_two_pass(x_list,n)

4.64 ms ± 53.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit ss_single_pass(x_list,n)

4.3 ms ± 73 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit ss_numerical_operations(x_list,n)

4 ms ± 67.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# Indeed, memory access is twice as slow in the two pass versus the one pass algorithms
(4.6-4)/(4.3-4)
# And also notice that the numerical operations themselves (~4 ms)
# are less optimized than the memory access operations (~0.3 ms)

2.0

In [ ]:
# naive two pass algorithm with `x_np_array` like 100x faster than with `x_list`
%timeit (x_np_array**2).sum() - x_np_array.sum()**2/len(x_np_array)

In [ ]:
# single pass `np.var()` "single pass recurrsive" algorithm is twice as slow...
%timeit x_np_array.var(ddof=0)*len(x_np_array)
# because it uses more numerical operations to attempt to reduce roundoff error

<a name="cell-operation-counting-fan"></a>

### The sequential summation versus the `fan` method ([Return to TOC](#cell-solving))

---

The `STA410_W24_Week4_Homework_AdditionVariance.ipynb` considered a sequential summation algorithm and the alternative `fan` algorithm. Convince yourself that these require the same number of additions. Now, consider the run times below. The difference is not due to a different number of additions since those is the same.  The difference is caused by the the fact that the `fan` requires memory in which to store the new list to pass to the next recursive call of the `fan` function. 

In [ ]:
x = np.linspace(1,2,2**15)

def ss(x):
  x_sum = 0
  for xi in x:
    x_sum += xi

def fan(x):
    '''assumes len(x)=2**p always'''
    if len(x)==1:
        return x
    return fan([x[j*2]+x[j*2+1] for j in range(int(len(x)/2))])    

In [ ]:
%timeit ss(x)

In [ ]:
%timeit fan(x)

<a name="cell-sovling-bigO"></a>

## 4.1.2 "Big O" Algorithmic Complexity ([Return to TOC](#cell-solving))

---

An algorithm requiring $f(n)$ operations to complete a problem with input size $n$ has ***Big $O$*** ***algorithmic complexity*** order $g(n)$, written as 

$$f(n) = O(g(n)) \quad \text{ if } \quad f(n) \leq M g(n) \quad \text{ for all $n>n_0$}$$

for strictly positive $g$ and some constant $M$ that is not a function of $n$.

The "***Big O***" ***algorithmic complexity*** of an algorithm measures the *proportional* computational demands of the algorithm as a function of the input size $n$ of the problem.

> While $f(n) = an^2+bn+c = O(n^2)$, the $O(n^2)$ only characterizes the computational characteristics of an algorithm in the context of very large $n$. 
 Specifying "***Big O***" ***computational complexity*** is pretty simple and generally just requires dropping lower order terms and ignoring proportinality coefficients. The actual runtime of an algorithm for a given input size $n$ depends on $a$ and the lower order terms $bn+c$ but these are not characterized by $O(n^2)$ since as $n$ grows large $an^2+bn+c< Mn^2$ for some large $M$. 
>
> $O(100n) = O(n) < O(n^2)$, but for $n=100$, the actual computational requirements for $f(n)=100n$ and $g(n)=n^2$ would be indistinguishable.  

<a name="cell-sovling-bigO-On-O1"></a>

### $O(n)$ and $O(1)$ ([Return to TOC](#cell-solving))

---

"***Big O***" ***algorithmic complexity*** only characterizes the long term cost of an algorithm as the input $n$ gets larger.  For example, the number of memmory reads for the two pass and single pass sum of squares have the same "***Big O***" ***algorithmic complexity*** 

$$SS_{2pass}^{\#memreads}(n) = 2n = O(n) \quad \text{ and } \quad SS_{1pass}^{\#memreads}(n) = n = O(n)$$

despite the fact that the former requires twice as many actual memory reading computations as the latter. 


The fact that an algorithm has a "***Big O***" ***algorithmic complexity*** $O(n)$ doesn't mean we know the specific computational requirements at a given problem input size $n$; rather, it characterizes that these computational requirements will scale ***linearly*** as a function of $n$.


> As we demonstrated, the memory access costs of the "two pass" algorithm are twice those of the "single pass" algrithm, depite both algorthms having "***Big O***" ***algorithmic complexity*** $O(n)$.
>
> Similarly, the the overall computational requirements of 
> 
> - `ss_two_pass(x,len(x))` and `ss_single_pass(x,len(x))`
>
> are about 100 times that of 
> 
> - `x.var(ddof=0)*len(x)` 
>
> which in turn requires about 2 times that of 
>
> - `lambda x: (x**2).sum() - x.sum()**2/len(x) # x an np.array`
> 
> and yet all of these algorithms have "***Big O***" ***algorithmic complexity*** $O(n)$.

> Another good example that is helpful for understanding the nature of "***Big O***" ***algorithmic complexity*** is the case of $O(1)$, which means that the computational requirements do not change as a function of the size of the input data.
>
> - An example of $O(1)$ is storage requirements **beyond** an input buffer needed to calculate a running mean, sum, or variance from streaming data (since these computations all admit "streaming" one pass calculations).
>    - An algorithm which simply needs to stream through an input buffer,  *even if it does so several times*, is called an ***online*** or **"out of core"** algorithm. This is why the "two pass" sum of squares algorithm was called an ***online*** algorithm.
> - Median calculations do not admit an $O(1)$ storage requirement. The storage requirements of a median calculation are $O(n)$ because to determine which data point is the median the complete data set must be sorted in memory. 
>    - An algorithm with $O(n)$ storage requirements is called a ***batch*** algorithm and requires constant access to the input data.
> - An ***online*** or **"out of core"** algorithm that requires only a *single* pass through an input buffer *and* only has an additional storage requirement of $O(1)$ is called a ***real-time*** algorithm.
>    - Variance calculations using only a single pass of the data are ***real-time*** algorithms; and, covariance matrix calculations also admit ***real-time*** forms; and, linear regression model fitting in fact also has a ***real-time*** form (which doesn't even requires any covariance matrix calculations). 

<a name="cell-sovling-bigO-On2-On3"></a>

### $O(n^2)$ and $O(n^3)$ ([Return to TOC](#cell-solving))

---

Examples of "***Big O***" ***algorithmic complexity*** $O(n^2)$ and $O(n^3)$ are matrix-vector multiplication and matrix-matrix multiplication, respectively. 

- matrix-vector multiplication $A_{n \times n} x_{n \times 1}$ requires $n^2$ multiplications and $n(n − 1)$ additions: $n^2 + n(n − 1) = O(n^2)$

  > $10,000^2 = 100,000,000$ might be computationally tractable; but, $n^2$ for larger $n$ quickly starts to look computationally unappealing...

- matrix-matrix multiplication $A_{n \times n} B_{n \times n}$ requires $n$ matrix-vector multiplication: $n\times(n^2 + n(n − 1)) = O(n^3)$

  > $1000^3 = 1,000,000,000$ *might* be computationally tractable; but, $n^3$ for larger $n$ immediately looks like a computational death trap...

In terms of the problem input size $n$, it becomes quickly apparent that the computational requirements of "***Big O***" ***algorithmic complexity*** $O(n^2)$ and $O(n^3)$ algorithms scale tremendously differently as $n$ increases, and that neither is particularly attractive as a function of $n$ relative to "***Big O***" ***algorithmic complexity*** $O(n)$.

<a name="cell-FPN-counting"></a>

#### A Counting Example with Convolutions ([Return to TOC](#cell-solving))

---

Determining $O(f(n))$ is simply a matter of defining what's being counted, and then carefully counting it. 

> Consider the following (***convolution***) algorithm $f(n)$ computing the distribution of $p(X_1+\cdots+X_n)$ for discrete $X_i \overset{i.i.d.}{\sim} p(X_i=x)$ with $x \in \{0, \cdots, K\}$:
>
> $$\begin{align*}
p(X_1+X_2 = s) = & {} \sum_{x=0}^{\min(s,K)} p(X_1=s-x) p(X_2=x)\\
p(X_1+X_2+X_3 = s) = & {} \sum_{x=0}^{\min(s,K)} p(X_1+X_2 = s-x) p(X_3=x)  \\
p(X_1+X_2+X_3+X_4 = s) = & {} \sum_{x=0}^{\min(s,K)} p(X_1+X_2+X_3 = s-x) p(X_4=x)  \\
\vdots  & {}
\end{align*}$$

For this example some possible counting targets might be 
- How many multiplications are required to define $p(X_1+\cdots+X_n)$?
- How many additional additions are required to define $p(X_1+\cdots+X_n)$?
- How much storage is required to compute $p(X_1+\cdots+X_n)$?

> To answer the first counting question about multiplication, note that at each stage 
>  1. $\quad p(X_1+X_2)$
>  2. $\quad p(X_1+X_2+X_3)$
>
>    $\quad\quad\quad\quad\vdots$
> 
> $\quad$k. $\quad p(X_1+\cdots+X_k = s)$ must be calculated for $s \in \{0, \cdots, kK\}$
> 
> and each 
> - $p(X_1+\cdots+X_k = s)$ calculation requires the sum of $\min(s,K)+1$ multiplications of $p(X_k)$ with the previous $p(X_1+\cdots+X_{k-1})$, i.e.,
>
>   $$\sum_{x=0}^{\min(s,K)} \underbrace{p(X_1+\cdots+X_{k-1} = s-x)}_{\text{can be cached as a time-space tradeoff}} p(X_k=x)$$
>
>   which can be cached (as a ***time-space tradeoff***) if the algorithm is approached in the natural sequential manner.
>
> Thus, the total number of multiplications required is 
>
> $$\begin{align*}
\overset{\text{stage}}{\sum_{k=2}^n} \overbrace{\sum_{s=0}^{kK} \big(\min(s,K)+1\big)}^{\text{stage $k$ multiplications}} & \leq {} \sum_{k=1}^n \sum_{s=0}^{kK} \big(K+1\big)\\
& = {} \sum_{k=1}^n (kK+1) (K+1)\\
O\left(\sum_{k=1}^n (kK+1) (K+1)\right) & = {} O\left(\sum_{k=1}^n kK^2+kK + K+1 \right)\\
& = {} O\left(\sum_{k=1}^n kK^2\right) = O\left( \frac{n(n+1)}{2}K^2\right)\\
& = {} O\left(n^2K^2\right)
\end{align*}$$

> There is an alternative algorithmic approach for this problem based on the ***moment generating function***
> 
> $$\begin{align*}
E\left[t^S\right] = {} & \sum_{s=0}^{nK} t^s p(S=s) \\
= E\left[t^{X_1 + \cdots + X_n}\right] = {} &  \underbrace{E\left[\prod_{i=1}^n t^{X_i}\right] = \prod_{i=1}^nE\left[ t^{X_i}\right]}_{\text{since } X_1, \cdots, X_n \text{ are independent}}\\  
= {} & \left(\sum_{x=0}^{K} t^x p(X_i=x) \right)^n\\ 
\end{align*}$$
> 
> which for $nK+1$ distinct values of $t$, results in $nK+1$ equations with $nK+1$ unknowns, i.e., $p(S=s)$ for $s=0,\cdots,nK$; however, solving a sytem of $nK+1$ equations is generally an $O(n^3K^3)$ computation, so this formulation alone will not improve upon the initial algorithm.   

*The example in this section is inspired by Keith Knight's STA410 [class notes](https://q.utoronto.ca/courses/244990/files?preview=18669454); however, rather than summing over the possible values of the accumulating random variable $(s)$, the summation in the formulation presented here is done only over all possible values of the newly contributed random variable $(x)$, which is an improved formulation since $O\left(K^2 n^2\right) < O\left(K^2 n^3\right)$;  indeed, the formulation achieves the same algorithmic complexity as a* ***fast Fourier transform*** *because in the discrete case the computation can be restricted to only the arithmetic necessary for the solution.*

<a name="cell-sovling-bigO-Onlogn"></a>

### $O(n\log(n)) \text{ and } O(n^{\log_2 7}) \approx O(n^{2.81})$ ([Return to TOC](#cell-solving))

---

"***Big O***" ***algorithmic complexity*** $O(n \log n)$ arises from  algorithms leveraging the ***divide and conquer*** nature of [recursion](https://www.google.com/search?q=recursion)

```python
divNconq_recursion(x):
    if stopping_criterion_met:
        return small_problem_answer
    # otherwise extract `half_of_x` and `rest_of_x` and
    return DivnConq_recursion(half_of_x) + DivnConq_recursion(rest_of_x)
```

which (recursively) reduces a problem into two separate problems, each of half the size as the original until they are small enough to solve rapidly. This results in computationally tractable $O(n\log n)$ algorithms which are feasible for large $n$ compared to $O(n^2)$. 

- An input of size $n$ be can recursively split in half $\log_2 n$ times since definitionally $2^{\log_2 n} = n$. At each stage of subsequent (recursive) splitting, all of the $n$ elements must still be accessed in order to determine the new splits.  So for each $2^{\log_2 n}$ splitting stage, $n$ input elements are accessed, and this is what leads to the $O(n\log n)$ "***Big O***" ***algorithmic complexity*** of ***divide and conquer*** recursion. 

> The famous ***Fast Fourier Transform*** 
> - $\tilde x = Ax, \;A_{jk} = e^{-k\left(\frac{j}{n}\right) 2\pi i}$
> - $x = A^{-1}\tilde x, \;A^{-1}_{jk} = \frac{1}{n}e^{k\left(\frac{j}{n}\right) 2\pi i}$
> is a special matrix-vector multiplication operation that can be computed in an $O(n \log n)$ recursive ***divide and conquer*** manner, as opposed to the typical $O(n^2)$ computation of standard matrix-vector multiplication.  Thus, if the problem being solved is in some way benefited by making the initial transformation, then the transformation itself will likely not be a limiting computational factor. 

> Strassen's divide and conquer matrix multiplication algorithm achieves $O(n^{\log_2 7}) \approx O(n^{2.81})$ which is a significant imporovement over $O(n^3)$ for moderate $n$.

Not all ***divide and conquer*** algorithms are necessarily $O(n \log n)$. The `fan` summation method from the [Week 2 Programming Assignment]() can be specified as the ***divide and conquer*** algorithm 

```python
def fansum_recursive(x):
  '''assumes len(x)=2*p always'''
  if len(x)==2:
    return sum(x)
  return fansum(x_odds)+fansum(x_evens)
```

which is $O(n \log n)$ as described above.  But this is because the recursive overhead is actually unnecessary and is equivalent to 


```python
def fansum_iterative(x):
  '''assumes len(x)=2*p always'''
  if len(x)==2:
    return sum(x)
  return fansum(x_odds+x_evens)
```

The `fan` method then is a ***divide and conquer*** algorithm which doesn't reduce a problem into two smaller problems that are half as small; but, rather, it reduces a problem of size $n$ to a single new problem of size $n/2$.

In [ ]:
def fansum_recursive(x):
  '''assumes len(x)=2*p always'''
  if len(x)==2:
    return sum(x)
  # slightly different but generally equivalent formulation
  return fansum_recursive(x[:int(len(x)/2)])+fansum_recursive(x[int(len(x)/2):])

def fansum_iterative(x):
  '''assumes len(x)=2*p always'''
  if len(x)==2:
    return sum(x)
   # slightly different but generally equivalent formulation   
  return fansum_iterative(x[:int(len(x)/2)]+x[int(len(x)/2):])

x = np.linspace(1,2,2**5)
# these are mathematically equivalent methods
np.isclose(fansum_recursive(x), fansum_iterative(x))
# but the iterative form is $O(n)$ and avoids "split into to half problems" 
# overhead entailed in the recursive divide and conquer version...

In [ ]:
# O(n log n) "split into to half problems"  divide and conquer
%timeit fansum_recursive(x)

In [ ]:
# O(n) "sum to reduce" direct iterative method with no log(n) overhead
%timeit fansum_iterative(x)

<a name="cell-sovling-direct-methods"></a>

## 4.1 Direct Methods for $X\hat \beta \approx y$ by solving for $\hat \beta$ in $\overset{A\, x}{X^TX \beta} = \overset{b}{X^T y}$ ([Return to TOC](#cell-solving))

---

## $\hat \beta$ solving $\;X^TX\hat \beta = X^Ty\;$ provides the ***least squares*** approximation $X\hat \beta \approx  y$ 


***Direct methods*** execute a set sequence of predefined deterministic computational steps to arrive at the exact solution (to the limits of available precision) to a computational problem. The various `sum` and `variance` computations considered in the `STA410_W24_Week4_Homework_AdditionVariance.ipynb` are examples of ***direct methods***. The ***Gram-Schmidt orthogonalization*** and ***Cholesky decomposition*** considered in the `STA410_W24_Week5_Extra_LinearAlgebraAlgorithms.ipynb` are examples of ***direct methods***. And the computation of the ***SVD*** is a ***direct methods***. While we have not seen exactly what the ***SVD*** algorithm is, suffice it to say that is is similar in character to the ***Gram-Schmidt orthogonalization*** and ***Cholesky decompositions***.

Here are some new examples of ***direct methods***

1. `np.linalg.inv(X.T@X) @ X^Ty # Analytical solution`
2. `np.linalg.solve(X.T@X, X^Ty) # Gaussian elimination`
3. `# QR decomposition based least squares estimates`

   `Q,R = np.linalg.qr(X); scipy.linalg.solve_triangular(R, Q.T@y)` 
4. `C = np.linalg.cholesky(X.T@X) # Cholesky decomposition based least squares estimates`

   `scipy.linalg.solve_triangular(C, scipy.linalg.solve_triangular(C, X.T@y, lower=True))`
5. `np.linalg.lstsq(X, y)  # SVD based least squares estimates` 

used to solve for the least squares solution $\hat \beta$ solving $X^TX\hat \beta = X^T y$ (or more generally for $x$ solving $Ax=b$ when a solution exists). These are ***direct methods*** because each each algorithmically computes the prescribed steps of a linear algebra prescription that computates $\hat \beta$.

> All of these methods (for $X_{n\times n}$ with $n>m$) are $O(nm^2)$; although, importantly, they will have different computational run times for a given $n$ and $m$ since $M$ in $f(n,m)\leq Mnm^2$ can be (and is) different for each algorithm.

<a name="cell-sovling-direct-methods-big0"></a>

## 4.1.0 "Big O" Algorithmic Complexity ([Return to TOC](#cell-solving))

---

The "***Big O***" ***algorithmic complexity*** for $n>m$ of all of the ***direct methods*** above is $O(nm^2)$. 

- `np.linalg.inv(X.T@X) @ X^Ty` is $(X^T@X)^{-1} (X_{n\times m}^Ty) = O(nm^2)$ since this term dominates the operations together $O(nm^2) + O(m^3) + O(nm) + O(m^2)$ 
  - `X.T@X` matrix-matrix multiplication is $X_{n\times m}^TX_{n\times m} = O(nm^2)$ 
  - `np.linalg.inv(A_mxm)` [matrix inversion](http://gregorygundersen.com/blog/2020/12/09/matrix-inversion/) and even [***upper triangular matrix*** inversion](https://math.stackexchange.com/questions/92068/fast-inversion-of-a-triangular-matrix) is $X_{n\times m}^Ty = O(m^3)$ 
  - `X.T@y` matrix-vector multiplication is $X_{n\times m}^Ty = O(nm)$
  - `Ainv_mxm @ b_mx1` is $O(m^2)$.

- `np.linalg.solve(X.T@X, X.T@y)` is again $O(nm^2)$ as $X_{n\times m}^TX_{n\times m} = O(nm^2)$ will again dominate $X_{n\times m}^Ty = O(nm)$ and the other required $O(m^3)$ and $O(m^2)$ oprations
  - `np.linalg.solve(A_mxm, b)` is $O(m^3)$ since (as shown previously) it requires the sequential application of 

    1. ***Gaussian elimination*** with a required operation count of $2 \times \left(\frac{m(m+1)(2m+1)}{6} - m + \frac{m(mm+1)}{2} - m \right) = O(m^3)$ 

    2. ***backward substitution*** with a required operation count of $m^2 = O(m^2)$ 

- The ***QR decomposition*** `Q,R = np.linalg.qr(X)` using [***Householder transformations***](https://math.stackexchange.com/questions/501018/what-is-the-operation-count-for-qr-factorization-using-householder-transformatio) which are more numerically accurate than the ***(modified) Graham-Schmidt procedure*** is $O(nm^2)$ 
  - `scipy.linalg.solve_triangular(R, Q.T@y)` is again $m^2 = O(m^2)$ 

- The ***Cholesky decomposition*** `C = np.linalg.cholesky(X.T@X)` is a ***Gaussian elimination*** like process of $O(m^3)$, but the prerequesite $X_{n\times m}^TX_{n\times m}$ computation is $O(nm^2)$

  - `scipy.linalg.solve_triangular(C, scipy.linalg.solve_triangular(C, X.T@y, lower=True))` are each $m^2 = O(m^2)$ 

- The [standard computation](https://www.mathworks.com/matlabcentral/answers/420057-what-is-the-complexity-of-matlab-s-implementation-of-svd) of the ***SVD*** [implemented](https://scicomp.stackexchange.com/questions/6979/how-is-the-svd-of-a-matrix-computed-in-practice) by `np.linalg.lstsq(X, y)` is $O(nm^2)$

  - The subsquent $\hat \beta = V (D^{+})^2 V^T X^T y$ is $O(\max(m^3,nm))$ depending on if $VV^T$ or $X^T y$ is more computationally expensive


